## Importando Bibliotecas

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [5]:
import pandas as pd

# Load the dataframe
df = pd.read_csv('C:/Users/Ani/Desktop/BVNNY/Master/TFM/Datasets/df_final.csv', sep=',')

# Display the first few rows of the dataframe
print(df.head())

# Display the columns of the dataframe
print(df.columns)

     Especie_planta  Toxic_Yes  Conocimiento_sobre_plantas_enc  \
0  Costilla de Adán        0.0                             1.0   
1  Costilla de Adán        1.0                             1.0   
2  Costilla de Adán        1.0                             1.0   
3  Costilla de Adán        0.0                             1.0   
4  Costilla de Adán        1.0                             1.0   

   Tiempo_de_dedicacion_enc  Vive_En_Clima_No_Seco  Tonos_Cálidos  \
0                       1.0                      1              0   
1                       2.0                      1              0   
2                       1.0                      1              0   
3                       1.0                      0              1   
4                       2.0                      1              0   

   Tonos_Fríos  Tonos_Me es indiferente  Vivienda_Adosado  \
0            1                        0                 0   
1            1                        0                 1   
2    

In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Mostrar las primeras filas del dataframe y las columnas
print(df.head())
print(df.columns)

# Paso 2: Preprocesamiento de Datos
# Convertir las características categóricas en números
label_encoders = {}
for column in df.columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

# Definir las características (X) y el objetivo (y)
X = df.drop('Sensación', axis=1)
y = df['Sensación']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 3: Construcción del Modelo de Red Neuronal
# Definir las entradas del modelo
input_layers = []
embedding_layers = []

# Crear embeddings para cada característica categórica
for column in X.columns:
    input_layer = Input(shape=(1,), name=column)
    embedding_size = min(50, len(df[column].unique()) // 2)
    embedding_layer = Embedding(input_dim=len(df[column].unique()), output_dim=embedding_size, name=f'{column}_embedding')(input_layer)
    embedding_layer = Flatten()(embedding_layer)
    
    input_layers.append(input_layer)
    embedding_layers.append(embedding_layer)

# Concatenar todos los embeddings
concat = Concatenate()(embedding_layers)

# Añadir capas densas
dense_1 = Dense(128, activation='relu')(concat)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

# Definir el modelo
model = Model(inputs=input_layers, outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

# Paso 4: Entrenamiento del Modelo
# Convertir los datos de entrada en una lista de arrays
X_train_list = [X_train[column] for column in X.columns]
X_test_list = [X_test[column] for column in X.columns]

# Entrenar el modelo
history = model.fit(
    X_train_list,
    y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test_list, y_test)
)

# Paso 5: Evaluar el Modelo
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test_list, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Paso 6: Hacer Recomendaciones
def recommend(item_features, model, num_recommendations=5):
    # Convertir las características en el formato adecuado
    item_features_list = [item_features[column] for column in X.columns]
    
    # Predecir la "Sensación" para los ítems
    predicted_ratings = model.predict(item_features_list)
    
    # Añadir las predicciones a las características de los ítems
    item_features['predicted_rating'] = predicted_ratings
    
    # Ordenar por la predicción y devolver las principales recomendaciones
    recommendations = item_features.sort_values(by='predicted_rating', ascending=False).head(num_recommendations)
    
    return recommendations

# Ejemplo de recomendación (usando las mismas características del dataset para ilustrar)
item_features = X_test.copy()
recommendations = recommend(item_features, model)
print(recommendations)


     Especie_planta  Toxic_Yes  Conocimiento_sobre_plantas_enc  \
0  Costilla de Adán        0.0                             1.0   
1  Costilla de Adán        1.0                             1.0   
2  Costilla de Adán        1.0                             1.0   
3  Costilla de Adán        0.0                             1.0   
4  Costilla de Adán        1.0                             1.0   

   Tiempo_de_dedicacion_enc  Vive_En_Clima_No_Seco  Tonos_Cálidos  \
0                       1.0                      1              0   
1                       2.0                      1              0   
2                       1.0                      1              0   
3                       1.0                      0              1   
4                       2.0                      1              0   

   Tonos_Fríos  Tonos_Me es indiferente  Vivienda_Adosado  \
0            1                        0                 0   
1            1                        0                 1   
2    

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Especie_planta      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Toxic_Yes           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conocimiento_sobre… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Tiempo_de_dedicaci… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Vive_En_Clima_No_S… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Tonos_Cálidos       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Tonos_Fríos         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Tonos_Me es         │ (None, 1)         │          0 │ -                 │
│ indiferente         │                   │            │                   │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Vivienda_Adosado    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Vivienda_Apartamen… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Vivienda_Chalet     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dificultad          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Ubicación           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Estatura            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Frecuencia_Riego    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Tolera_mediterráne… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 12,412 (48.48 KB)

 Trainable params: 12,412 (48.48 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


ValueError: Exception encountered when calling Functional.call().

[1m'Tolera_mediterráneo_seco_embedding' is not a valid scope name. A scope name has to match the following pattern: ^[A-Za-z0-9_.\\/>-]*$[0m

Arguments received by Functional.call():
  • inputs=('tf.Tensor(shape=(None, 1), dtype=int32)', 'tf.Tensor(shape=(None, 1), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 1), dtype=int64)')